In [10]:
# Import required libraries
import PyPDF2
import textract
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import docx2txt

resume = 'Chong Fai Edward Lam Resume.docx'

job_description = 'job description - pt form.docx'

def read_input_file(filename):

    if 'pdf' in filename:
        # Open pdf file
        pdfFileObj = open('sample_resume_.pdf','rb')

        # Read file
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

        # Get total number of pages
        num_pages = pdfReader.numPages

        # Initialize a count for the number of pages
        count = 0

        # Initialize a text empty etring variable
        text = ""

        # Extract text from every page on the file
        while count < num_pages:
            pageObj = pdfReader.getPage(count)
            count +=1
            text += pageObj.extractText()
        
    elif 'doc' in filename:

        # Passing docx file to process function
        text = docx2txt.process(filename)
        
    return text


In [11]:
def text_cleansing(text):
    # Convert all strings to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+','',text)

    # Remove punctuation
    return text.translate(str.maketrans('','',string.punctuation))

In [21]:
resume_txt = text_cleansing(read_input_file(resume))
jd_txt = text_cleansing(read_input_file(job_description))

In [13]:
jd_txt

'database related skills \n\nmysql \n\nms server sql\n\npostgresql\n\nweb application \n\nnodejs \n\ndjango \n\nhtml \n\ncss \n\njavascript \n\najax \n\nphp \n\nreactjs\n\nscraping experience\n\naws related services\n\ns\n\nlambda\n\nec\n\nrds\n\nglue\n\nredshift\n\nairflow\n\nvisualization skills\n\ntableau\n\npowerbi\n\nsoftware development in python\n\nproficiency in data warehouse concepts and data engineer related activities such as data mapping data cleansing and ingestion to data warehouse datamart generation using stored procedure etc\n\ndata mapping\n\ndatamart\n\ndata warehouse\n\ncomfortable in working in agile development cycle keeping track of jira tickets contacting business for requirement confirmation overseeing the development and contributing in scrum meetings\n\nagile\n\njira\n\nscrum'

In [ ]:
#function that does phrase matching and builds a candidate profile
def create_profile(file):
    text = pdfextract(file) 
    text = str(text)
    text = text.replace("\\n", "")
    text = text.lower()
    #below is the csv where we have all the keywords, you can customize your own
    keyword_dict = pd.read_csv('D:/NLP_Resume/resume/template_new.csv')
    stats_words = [nlp(text) for text in keyword_dict['Statistics'].dropna(axis = 0)]
    NLP_words = [nlp(text) for text in keyword_dict['NLP'].dropna(axis = 0)]
    ML_words = [nlp(text) for text in keyword_dict['Machine Learning'].dropna(axis = 0)]
    DL_words = [nlp(text) for text in keyword_dict['Deep Learning'].dropna(axis = 0)]
    R_words = [nlp(text) for text in keyword_dict['R Language'].dropna(axis = 0)]
    python_words = [nlp(text) for text in keyword_dict['Python Language'].dropna(axis = 0)]
    Data_Engineering_words = [nlp(text) for text in keyword_dict['Data Engineering'].dropna(axis = 0)]

    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('Stats', None, *stats_words)
    matcher.add('NLP', None, *NLP_words)
    matcher.add('ML', None, *ML_words)
    matcher.add('DL', None, *DL_words)
    matcher.add('R', None, *R_words)
    matcher.add('Python', None, *python_words)
    matcher.add('DE', None, *Data_Engineering_words)
    doc = nlp(text)
    
    d = []  
    matches = matcher(doc)
    for match_id, start, end in matches:
        rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
        span = doc[start : end]  # get the matched slice of the doc
        d.append((rule_id, span.text))      
    keywords = "\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())
    
    ## convertimg string of keywords to dataframe
    df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
    df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
    df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
    df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
    df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
    
    base = os.path.basename(file)
    filename = os.path.splitext(base)[0]
       
    name = filename.split('_')
    name2 = name[0]
    name2 = name2.lower()
    ## converting str to dataframe
    name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])
    
    dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)

    return(dataf)

In [27]:
jd_txt_list = jd_txt.split("\n")
jd_txt_list = list(dict.fromkeys(jd_txt_list))

matching_word = []
not_matching_word = []

for word in jd_txt_list:
    if word in resume_txt:
        matching_word.append(word)
    else:
        not_matching_word.append(word)
        

In [28]:
matching_word

['',
 's',
 'lambda',
 'ec',
 'glue',
 'redshift',
 'airflow',
 'tableau',
 'data mapping',
 'data warehouse',
 'scrum']

In [29]:
not_matching_word

['database related skills ',
 'mysql ',
 'ms server sql',
 'postgresql',
 'web application ',
 'nodejs ',
 'django ',
 'html ',
 'css ',
 'javascript ',
 'ajax ',
 'php ',
 'reactjs',
 'scraping experience',
 'aws related services',
 'rds',
 'visualization skills',
 'powerbi',
 'software development in python',
 'proficiency in data warehouse concepts and data engineer related activities such as data mapping data cleansing and ingestion to data warehouse datamart generation using stored procedure etc',
 'datamart',
 'comfortable in working in agile development cycle keeping track of jira tickets contacting business for requirement confirmation overseeing the development and contributing in scrum meetings',
 'agile',
 'jira']